In [1]:
import cv2
import time


In [2]:
import pandas as pd

In [3]:
import numpy as np

In [ ]:
input_images3=[]
target3=[]
count2=0
capture_video=cv2.VideoCapture(0)
for i in range(2):
    no_of_images3=0
#         capture_video=cv2.VideoCapture(0)
#         time.sleep(5)
    input("enter to start")
    while True:
        _,image1=capture_video.read(0)
        if not _:
            break
        gray5=cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
        haarcascade=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        detected_objects=haarcascade.detectMultiScale(gray5,1.2,9)
             #if len(detected_objects)>1:
                #continue
            #else:
                
        for bbox in detected_objects:
            x,y,w,h=bbox
            cv2.rectangle(image1,(x,y),(x+w,y+h),(0,0,255),3)#making a rectangle over one of detected faces.
            crop3=gray5[y:y+h,x:x+w]#cropping just the face.
            crop3=cv2.resize(crop3,(100,100))
            input_images3.append(crop3)
            target3.append(count2)
            no_of_images3+=1
            cv2.imshow("cropped",crop3)
            key=cv2.waitKey(1)
            cv2.imshow("image",image1)
#             if(cv2.waitKey(1) & 0xFF==ord('q')):
#                 break
#             if cv2.getWindowProperty("Generated video", 4) < 1:
#                 break
        if no_of_images3==500:
            break  
#             capture_video.release()
#             cv2.destroyAllWindows()
    count2+=1
    cv2.destroyAllWindows()
    print(input_images3)
    print(_)
    print(count2)
capture_video.release()


#BELOW:
    #normalization
#     input_images=input_images/255
    #reshaping to keep every pixel value inside an array
#     input_images = input_images.reshape(1000,100,100,1)


enter to start


In [ ]:
len(input_images3)

In [ ]:
len(target3)

In [ ]:
input_images3_normalized=np.array(input_images3)/255.0
input_images3_normalized

In [ ]:
input_images3_normalized_reshaped=input_images3_normalized.reshape(1000,100,100,1)

In [ ]:
input_images3_normalized_reshaped

In [ ]:
from keras.utils import np_utils

In [ ]:
target3=np_utils.to_categorical(target3)

In [ ]:
target3[:100]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(input_images3_normalized_reshaped,target3)

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Conv2D,MaxPooling2D,Dropout,Flatten
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(300, (3,3), input_shape=(100, 100, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dense(50,activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
cp=ModelCheckpoint('best_one',verbose=0,save_best_only=True)
model.fit(X_train,Y_train,epochs=5,callbacks=[cp],validation_split=0.2)

In [ ]:
from keras.models import load_model
model=load_model('best_one')

In [ ]:
import os
import imghdr
import smtplib
from email.message import EmailMessage
capture_video=cv2.VideoCapture(0)
img_counter=0

while True:
    _,image1=capture_video.read(0)
    gray6=cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
    haarcascade=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    detected_objects=haarcascade.detectMultiScale(gray6,1.2,9)
    for bbox in detected_objects:
        x,y,w,h=bbox
        cv2.rectangle(image1,(x,y),(x+w,y+h),(0,0,255),3)
        crop3=gray6[y:y+h,x:x+w]
        cropped_image=cv2.resize(crop3,(100,100))
        normalized_image=cropped_image/255
        reshaped_face=np.reshape(normalized_image, (1,100,100,1))
        result=model.predict(reshaped_face)[0]
        if result[0]>0.20:
            img_counter+=1
            
            if img_counter==1:
                _,frame=capture_video.read()
                img_name = "Intruder{}.png".format(img_counter)
                cv2.imwrite(img_name, frame)
                user = os.environ.get('EMAIL')
                password = os.environ.get('PYTHONPASSWORD')
                msg=EmailMessage()
                msg['Subject']="Intruder Alert!!"
                msg['From']=user
                msg['To']=user
                msg.set_content("Intruder identified at your door, suspect's photo is attached below!")
                with open(img_name,'rb') as f:
                    file_data=f.read()
                    file_type=imghdr.what(img_name)
                    file_name=img_name
                #     print(file_type)
                msg.add_attachment(file_data,maintype='image',subtype=file_type,filename=file_name)
                with smtplib.SMTP_SSL('smtp.gmail.com',465) as smtp:
                    smtp.login(user,password)
                    smtp.send_message(msg)
            cv2.rectangle(image1,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.putText(image1,"Intruder",(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
#             continue
        if result[1]>0.80:
            cv2.rectangle(image1,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(image1,"Innocent",(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow("image",image1)
    key=cv2.waitKey(1)
    if key==27:
        break  
cv2.destroyAllWindows()
capture_video.release()
